In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
from tqdm.notebook import tqdm
import pandas as pd
import re


BASE = "models/gemma3_1b"
ADAPTER = "models/gemma3_1b_text2sql_lora"

In [42]:
tokenizer = AutoTokenizer.from_pretrained(ADAPTER)
base_model = AutoModelForCausalLM.from_pretrained(BASE, device_map="auto")

In [43]:
model = PeftModel.from_pretrained(base_model, ADAPTER)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForCausalLM(
      (model): Gemma3TextModel(
        (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma3DecoderLayer(
            (self_attn): Gemma3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1152, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1152, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
        

In [4]:
def format_test_example(row) -> str:
    return f"""<|system|>
Ты — интеллектуальный текстовый помощник, преобразующий команды пользователя в корректные SQL-запросы на языке SQLite.

Ты всегда работаешь с таблицей `transactions`, имеющей следующую структуру:

- id (INTEGER, PRIMARY KEY)
- user_id (TEXT)
- type (TEXT: 'income' или 'expense')
- category (TEXT)
- amount (REAL)
- date (TIMESTAMP, по умолчанию date('now', 'localtime'))

Описание:
- `income` — это доход
- `expense` — это трата

Твоя задача — по команде пользователя с его ID сгенерировать корректный SQL-запрос к этой таблице.

Важно:
- Используй только SQLite-синтаксис, без пояснений.
- Не пиши комментарии.
- Не используй несуществующие поля (например, `note`).
- Поле `date` можно указывать через date('now', '-N day') или опустить (будет по умолчанию).

Пример:
---
Пользователь с ID user_1 дал команду:
"Добавь трату 500 рублей на еду вчера"

Ответ:
INSERT INTO transactions (user_id, type, category, amount, date)
VALUES ('user_1', 'expense', 'еда', 500, date('now', '-1 day'));
---
</s>
<|user|>
Пользователь с ID {row["user_id"]} дал команду:
"{row["user_command"]}"
</s>
<|assistant|>
"""

In [33]:
def clean_sql(text):
    """
    Удаляет обёртки ```sql ... ``` и специальные токены вроде </s>, <|...|>.
    """
    if not isinstance(text, str):
        return ""

    # Убираем блоки ```sql и ```
    text = re.sub(r"```sql\s*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"```", "", text)

    # Убираем спецтокены LLM
    text = re.sub(r"</s>", "", text)
    text = re.sub(r"<\|.*?\|>", "", text)

    return text.strip()

In [34]:
def generate_sql_from_df(test_df, model, tokenizer, max_new_tokens=128):
    predicted_sqls = []

    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        prompt = format_test_example(row)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.0,
                pad_token_id=tokenizer.eos_token_id
            )

        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        sql = decoded.split("<|assistant|>")[-1].strip()
        predicted_sqls.append(clean_sql(sql))

    test_df["predicted_sql"] = predicted_sqls
    return test_df

In [25]:
def predict_sql(test_df, model, tokenizer, max_new_tokens=1024, temperature=0.0):
    predicted_sql = []
    model.eval()

    for _, row in tqdm(test_df.iterrows(), desc="Генерация SQL", total=len(test_df)):
        prompt = format_test_example(row)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=False
            )


        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Извлекаем только SQL часть (после <|assistant|>)
        if "<|assistant|>" in decoded:
            predicted = decoded.split("<|assistant|>")[-1].strip()
        else:
            predicted = decoded.strip()

        predicted_sql.append(predicted)

    return predicted_sql

In [10]:
test_df = pd.read_csv("data/fin_ass_test.csv")

In [35]:
temp = test_df.head(5).copy()

In [36]:
temp = generate_sql_from_df(temp, model, tokenizer)

  0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [38]:
temp

,user_id,user_command,gold_sql,predicted_sql
0,user_4,Сколько я потратил на развлечения за последние...,SELECT SUM(amount) FROM transactions\nWHERE us...,SELECT SUM(amount
1,user_3,Сколько я потратил на одежда за последние 30 д...,SELECT SUM(amount) FROM transactions\nWHERE us...,SELECT SUM(amount) FROM transactions\nWHERE us...
2,user_5,Покажи мои расходы по категориям за последние ...,"SELECT category, SUM(amount) FROM transactions...","SELECT category, SUM(amount) FROM transactions..."
3,user_4,Покажи мои траты на зарплата между '2024-01-01...,SELECT * FROM transactions\nWHERE user_id = 'u...,"INSERT INTO transactions (user_id, type, categ..."
4,user_2,Сколько я потратил на продукты за последние 1 ...,SELECT SUM(amount) FROM transactions\nWHERE us...,SELECT SUM(amount) FROM transactions


In [39]:
print(temp.iloc[1]["predicted_sql"])

SELECT SUM(amount) FROM transactions
WHERE user_id = 'user_3' AND type = 'expense' AND category = 'одежда' AND date >= date('now', '-30 days');


In [40]:
print(temp.iloc[1]["gold_sql"])

SELECT SUM(amount) FROM transactions
WHERE user_id = 'user_3' AND type = 'expense' AND category = 'одежда' AND date >= date('now', '-30 days');


In [28]:
preds = predict_sql(temp, model, tokenizer)

Генерация SQL:   0%|          | 0/1 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [30]:
temp

,user_id,user_command,gold_sql
0,user_4,Сколько я потратил на развлечения за последние...,SELECT SUM(amount) FROM transactions\nWHERE us...
